## Importando bibliotecas e dados

In [1]:
import pandas as pd
import numpy as np
from pycaret.classification import setup, compare_models, models, create_model
from pycaret.classification import predict_model, tune_model, plot_model
from pycaret.classification import evaluate_model, finalize_model, save_model
from pycaret.classification import load_model
from pycaret.utils import check_metric
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',None)

In [2]:
train_raw_data=pd.read_csv('data/train.csv')
test_raw_data=pd.read_csv('data/test.csv')

In [3]:
train_raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_raw_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
trainrow=train_raw_data.shape[0]
trainrow

891

In [6]:
testrow=test_raw_data.shape[0]
testrow

418

In [7]:
y_train=train_raw_data['Survived'].copy()

In [8]:
train_raw_data=train_raw_data.drop(['Survived'],1)

In [9]:
combine=pd.concat([train_raw_data,test_raw_data])
combine.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [10]:
print(combine.shape)
print(train_raw_data.shape)
print(test_raw_data.shape)

(1309, 11)
(891, 11)
(418, 11)


In [11]:
combine.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [12]:
combine['Embarked']=combine['Embarked'].fillna(combine['Embarked'].value_counts().index[0])

In [13]:
combine['Cabin'].value_counts()

C23 C25 C27        6
B57 B59 B63 B66    5
G6                 5
F2                 4
D                  4
                  ..
F E57              1
D50                1
A31                1
A10                1
C51                1
Name: Cabin, Length: 186, dtype: int64

In [14]:
combine['Cabin']=combine['Cabin'].fillna('U')

In [15]:
combine['Cabin'].value_counts()

U                  1014
C23 C25 C27           6
G6                    5
B57 B59 B63 B66       5
B96 B98               4
                   ... 
F E57                 1
D50                   1
A31                   1
A10                   1
C51                   1
Name: Cabin, Length: 187, dtype: int64

In [16]:
combine['Cabin']=combine['Cabin'].astype(str).str[0]

In [17]:
combine.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,U,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,U,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,U,S


In [18]:
combine.loc[combine['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,U,S


In [19]:
combine['Fare']=combine['Fare'].fillna(combine.loc[(combine['Pclass']==3) & 
                                                   (combine['Sex']=="male") & 
                                                   (combine['Age']<65) & 
                                                   (combine['Age']>55)].dropna()['Fare'].mean())

In [20]:
combine.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            263
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin            0
Embarked         0
dtype: int64

In [21]:
passengerids=test_raw_data['PassengerId']
passengerids

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [22]:
combine=combine.drop(['PassengerId','Ticket'],1)

In [23]:
combine.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,U,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,U,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,U,S


In [24]:
combine['familysize']=combine['SibSp']+combine['Parch']+1
combine.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,familysize
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,U,S,2
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,C,2
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,U,S,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C,S,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,U,S,1


In [25]:
combine['Title'] = combine.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
combine.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,familysize,Title
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,U,S,2,Mr
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,C,2,Mrs
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,U,S,1,Miss
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C,S,2,Mrs
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,U,S,1,Mr


In [26]:
combine=combine.drop(['Name'],1)
combine.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,familysize,Title
0,3,male,22.0,1,0,7.2500,U,S,2,Mr
1,1,female,38.0,1,0,71.2833,C,C,2,Mrs
2,3,female,26.0,0,0,7.9250,U,S,1,Miss
3,1,female,35.0,1,0,53.1000,C,S,2,Mrs
4,3,male,35.0,0,0,8.0500,U,S,1,Mr


In [27]:
combine=combine.drop(['SibSp','Parch'],1)
combine.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,familysize,Title
0,3,male,22.0,7.2500,U,S,2,Mr
1,1,female,38.0,71.2833,C,C,2,Mrs
2,3,female,26.0,7.9250,U,S,1,Miss
3,1,female,35.0,53.1000,C,S,2,Mrs
4,3,male,35.0,8.0500,U,S,1,Mr


In [28]:
combine['Sex']=combine['Sex'].map({'male':0,'female':1})
combine.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,familysize,Title
0,3,0,22.0,7.2500,U,S,2,Mr
1,1,1,38.0,71.2833,C,C,2,Mrs
2,3,1,26.0,7.9250,U,S,1,Miss
3,1,1,35.0,53.1000,C,S,2,Mrs
4,3,0,35.0,8.0500,U,S,1,Mr


In [29]:
print(combine.Pclass.unique())
print(combine.Sex.unique())

[3 1 2]
[0 1]


In [30]:
for i in range(0,2):
    for j in range(0,3):
        print(i,j+1)
        temp_dataset=combine[(combine['Sex']==i) & (combine['Pclass']==j+1)]['Age'].dropna()
        print(temp_dataset)
        combine.loc[(combine.Age.isnull()) & (combine.Sex==i) & (combine.Pclass==j+1),'Age']=int(temp_dataset.median())

0 1
6      54.0
23     28.0
27     19.0
30     40.0
34     28.0
       ... 
378    55.0
390    23.0
403    17.0
404    43.0
407    50.0
Name: Age, Length: 151, dtype: float64
0 2
20     35.0
21     34.0
33     66.0
70     32.0
72     21.0
       ... 
387    57.0
393    47.0
401    38.0
405    20.0
406    23.0
Name: Age, Length: 158, dtype: float64
0 3
0      22.0
4      35.0
7       2.0
12     20.0
13     39.0
       ... 
394    29.0
396    24.0
398    22.0
399    31.0
415    38.5
Name: Age, Length: 349, dtype: float64
1 1
1      38.0
3      35.0
11     58.0
52     49.0
61     38.0
       ... 
397    48.0
400    30.0
402    22.0
411    37.0
414    39.0
Name: Age, Length: 133, dtype: float64
1 2
9      14.0
15     55.0
41     27.0
43      3.0
53     29.0
       ... 
330    48.0
349    31.0
361    24.0
362    31.0
385    24.0
Name: Age, Length: 103, dtype: float64
1 3
2      26.0
8      27.0
10      4.0
14     14.0
18     31.0
       ... 
367    22.0
376    22.0
383    19.0
409     3.0
4

In [31]:
combine.isnull().sum()

Pclass        0
Sex           0
Age           0
Fare          0
Cabin         0
Embarked      0
familysize    0
Title         0
dtype: int64

In [32]:
combine.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,familysize,Title
0,3,0,22.0,7.2500,U,S,2,Mr
1,1,1,38.0,71.2833,C,C,2,Mrs
2,3,1,26.0,7.9250,U,S,1,Miss
3,1,1,35.0,53.1000,C,S,2,Mrs
4,3,0,35.0,8.0500,U,S,1,Mr


In [33]:
combine_checkpoint=combine.copy()
combine.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,familysize,Title
0,3,0,22.0,7.2500,U,S,2,Mr
1,1,1,38.0,71.2833,C,C,2,Mrs
2,3,1,26.0,7.9250,U,S,1,Miss
3,1,1,35.0,53.1000,C,S,2,Mrs
4,3,0,35.0,8.0500,U,S,1,Mr


In [34]:
combine=combine_checkpoint.copy()
combine.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,familysize,Title
0,3,0,22.0,7.2500,U,S,2,Mr
1,1,1,38.0,71.2833,C,C,2,Mrs
2,3,1,26.0,7.9250,U,S,1,Miss
3,1,1,35.0,53.1000,C,S,2,Mrs
4,3,0,35.0,8.0500,U,S,1,Mr


In [35]:
combine['Age_Band']=pd.cut(combine['Age'],5)
combine['Age_Band'].unique()

[(16.136, 32.102], (32.102, 48.068], (48.068, 64.034], (0.0902, 16.136], (64.034, 80.0]]
Categories (5, interval[float64]): [(0.0902, 16.136] < (16.136, 32.102] < (32.102, 48.068] < (48.068, 64.034] < (64.034, 80.0]]

In [36]:
combine.loc[(combine['Age']<=16.136),'Age']=1
combine.loc[(combine['Age']>16.136) & (combine['Age']<=32.102),'Age']=2
combine.loc[(combine['Age']>32.102) & (combine['Age']<=48.068),'Age']=3
combine.loc[(combine['Age']>48.068) & (combine['Age']<=64.034),'Age']=4
combine.loc[(combine['Age']>64.034) & (combine['Age']<=80.),'Age']=5
combine['Age'].unique()

array([2., 3., 4., 1., 5.])

In [37]:
combine=combine.drop(['Age_Band'],1)


In [38]:
combine.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,familysize,Title
0,3,0,2.0,7.2500,U,S,2,Mr
1,1,1,3.0,71.2833,C,C,2,Mrs
2,3,1,2.0,7.9250,U,S,1,Miss
3,1,1,3.0,53.1000,C,S,2,Mrs
4,3,0,3.0,8.0500,U,S,1,Mr


In [39]:
combine['Fare_Band']=pd.cut(combine['Fare'],3)
combine['Fare_Band'].unique()

[(-0.512, 170.776], (170.776, 341.553], (341.553, 512.329]]
Categories (3, interval[float64]): [(-0.512, 170.776] < (170.776, 341.553] < (341.553, 512.329]]

In [40]:
combine.loc[(combine['Fare']<=170.776),'Fare']=1
combine.loc[(combine['Fare']>170.776) & (combine['Fare']<=341.553),'Fare']=2
combine.loc[(combine['Fare']>341.553) & (combine['Fare']<=512.329),'Fare']=3
combine=combine.drop(['Fare_Band'],1)

In [41]:
combine.head()

,Pclass,Sex,Age,Fare,Cabin,Embarked,familysize,Title
0,3,0,2.0,1.0,U,S,2,Mr
1,1,1,3.0,1.0,C,C,2,Mrs
2,3,1,2.0,1.0,U,S,1,Miss
3,1,1,3.0,1.0,C,S,2,Mrs
4,3,0,3.0,1.0,U,S,1,Mr


In [42]:
combine=pd.get_dummies(columns=['Pclass','Sex','Cabin','Embarked','Title','Age','Fare'],data=combine)
combine.head()

,familysize,Pclass_1,Pclass_2,Pclass_3,Sex_0,Sex_1,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S,Title_Capt,Title_Col,Title_Countess,Title_Don,Title_Dona,Title_Dr,Title_Jonkheer,Title_Lady,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Age_1.0,Age_2.0,Age_3.0,Age_4.0,Age_5.0,Fare_1.0,Fare_2.0,Fare_512.3292
0,2,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0
1,2,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,2,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
4,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0


In [43]:
x_train=combine.iloc[:trainrow]
x_test=combine.iloc[trainrow:]
x = x_train.copy()
x['survived'] = y_train
x.head()

,familysize,Pclass_1,Pclass_2,Pclass_3,Sex_0,Sex_1,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U,Embarked_C,Embarked_Q,Embarked_S,Title_Capt,Title_Col,Title_Countess,Title_Don,Title_Dona,Title_Dr,Title_Jonkheer,Title_Lady,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Age_1.0,Age_2.0,Age_3.0,Age_4.0,Age_5.0,Fare_1.0,Fare_2.0,Fare_512.3292,survived
0,2,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
1,2,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
2,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1
3,2,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
4,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0


In [44]:
x_train.shape

(891, 44)

# AutoML

In [45]:
clf = setup(data=x, target='survived')

Setup Succesfully Completed!


,Description,Value
0,session_id,7904
1,Target Type,Binary
2,Label Encoded,"0: 0, 1: 1"
3,Original Data,"(891, 45)"
4,Missing Values,False
5,Numeric Features,43
6,Categorical Features,1
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [46]:
best = compare_models(sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Gradient Boosting Classifier,0.8188,0.8817,0.7072,0.7977,0.7492,0.6083,0.6115,0.0954
1,CatBoost Classifier,0.8220,0.8816,0.7197,0.7987,0.7565,0.6169,0.6196,1.8035
2,Logistic Regression,0.8251,0.8790,0.7616,0.7799,0.7698,0.6289,0.6299,0.0167
3,Linear Discriminant Analysis,0.8300,0.8716,0.7699,0.7843,0.7768,0.6395,0.6399,0.0106
4,Ada Boost Classifier,0.8155,0.8705,0.7699,0.7582,0.7630,0.6121,0.6132,0.0902
5,Light Gradient Boosting Machine,0.8268,0.8668,0.7199,0.8104,0.7615,0.6262,0.6299,0.0741
6,Extreme Gradient Boosting,0.8284,0.8599,0.7196,0.8124,0.7625,0.6290,0.6324,0.1408
7,Random Forest Classifier,0.7883,0.8595,0.6650,0.7540,0.7063,0.5418,0.5448,0.0226
8,K Neighbors Classifier,0.8074,0.8389,0.6862,0.7880,0.7312,0.5826,0.5881,0.0059
9,Extra Trees Classifier,0.8027,0.8198,0.6737,0.7828,0.7232,0.5715,0.5762,0.1092


In [47]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors.KNeighborsClassifier,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model.SGDClassifier,True
rbfsvm,SVM - Radial Kernel,sklearn.svm.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process.GPC,False
mlp,MLP Classifier,sklearn.neural_network.MLPClassifier,False
ridge,Ridge Classifier,sklearn.linear_model.RidgeClassifier,True


In [48]:
dt = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7619,0.8072,0.7083,0.6800,0.6939,0.4992,0.4995
1,0.8254,0.8670,0.7917,0.7600,0.7755,0.6328,0.6331
2,0.8571,0.8878,0.7917,0.8261,0.8085,0.6947,0.6951
3,0.7903,0.8673,0.7500,0.7200,0.7347,0.5615,0.5618
4,0.8548,0.8975,0.7917,0.8261,0.8085,0.6917,0.6921
5,0.9355,0.9200,0.8750,0.9545,0.9130,0.8619,0.8640
6,0.8226,0.8717,0.7500,0.7826,0.7660,0.6232,0.6236
7,0.7742,0.8898,0.6250,0.7500,0.6818,0.5090,0.5141
8,0.8065,0.8531,0.7500,0.7500,0.7500,0.5921,0.5921
9,0.8226,0.9287,0.7826,0.7500,0.7660,0.6232,0.6236


In [49]:
tuned_dt = tune_model(dt, optimize='Accuracy')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7619,0.7975,0.7083,0.6800,0.6939,0.4992,0.4995
1,0.8254,0.8670,0.7917,0.7600,0.7755,0.6328,0.6331
2,0.8571,0.8953,0.7917,0.8261,0.8085,0.6947,0.6951
3,0.7903,0.8783,0.7500,0.7200,0.7347,0.5615,0.5618
4,0.8710,0.8986,0.7917,0.8636,0.8261,0.7238,0.7256
5,0.9194,0.9178,0.8750,0.9130,0.8936,0.8287,0.8292
6,0.8226,0.8750,0.7500,0.7826,0.7660,0.6232,0.6236
7,0.8065,0.8975,0.7083,0.7727,0.7391,0.5857,0.5871
8,0.8065,0.8553,0.7500,0.7500,0.7500,0.5921,0.5921
9,0.8226,0.9253,0.7826,0.7500,0.7660,0.6232,0.6236


In [50]:
evaluate_model(tuned_dt)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [51]:
predict_model(tuned_dt)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.8097,0.8421,0.7282,0.7653,0.7463,0.5942,0.5947


,Pclass_1,Pclass_2,Pclass_3,Sex_0,Sex_1,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,...,familysize_2,familysize_3,familysize_4,familysize_5,familysize_6,familysize_7,familysize_8,survived,Label,Score
0,0,0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.1242
1,0,1,0,0,1,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1,0.7884
2,0,0,1,1,0,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0576
3,0,0,1,0,1,0,0,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0.7698
4,0,0,1,0,1,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0.1035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,0,0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.1242
264,0,0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0785
265,0,0,1,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0785
266,0,1,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.1758


In [52]:
final_dt = finalize_model(tuned_dt)

In [53]:
predictions = predict_model(final_dt, data=x_test)
predictions

,familysize,Pclass_1,Pclass_2,Pclass_3,Sex_0,Sex_1,Cabin_A,Cabin_B,Cabin_C,Cabin_D,...,Age_1.0,Age_2.0,Age_3.0,Age_4.0,Age_5.0,Fare_1.0,Fare_2.0,Fare_512.3292,Label,Score
0,1,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0.0720
1,2,0,0,1,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,1,0.5615
2,1,0,1,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0.1302
3,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0.0803
4,3,0,0,1,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,0.7252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0.0803
414,1,1,0,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0.9494
415,1,0,0,1,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0.0509
416,1,0,0,1,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0.0803


In [54]:
final = test_raw_data[['PassengerId']]
final['Survived'] = predictions['Label']
final

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [55]:
final.to_csv('predictions.csv', index=False)

In [56]:
a = pd.read_csv('predictions.csv')
a

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


# Deep Learning

In [57]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.model_selection import train_test_split

In [58]:
x = x_train.copy()
y = y_train.copy()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
y_train_categorical = tf.keras.utils.to_categorical(y_train)
y_test_categorical = tf.keras.utils.to_categorical(y_test)
x_train.shape

(712, 44)

In [59]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,
                          input_dim=44,
                          kernel_initializer='normal',
                          activation='relu'),
    tf.keras.layers.Dense(88, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

In [60]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [61]:
history = model.fit(x_train, y_train, epochs=230, batch_size=92, validation_data=(x_test, y_test), verbose=1)

Epoch 1/230
8/8 [==============================] - 0s 18ms/step - loss: 0.6660 - accuracy: 0.6096 - val_loss: 0.6390 - val_accuracy: 0.6201
Epoch 2/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6313 - accuracy: 0.6138 - val_loss: 0.6018 - val_accuracy: 0.6257
Epoch 3/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5927 - accuracy: 0.6264 - val_loss: 0.5654 - val_accuracy: 0.6983
Epoch 4/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5537 - accuracy: 0.7163 - val_loss: 0.5266 - val_accuracy: 0.7654
Epoch 5/230
8/8 [==============================] - 0s 14ms/step - loss: 0.5170 - accuracy: 0.7893 - val_loss: 0.4893 - val_accuracy: 0.7877
Epoch 6/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4809 - accuracy: 0.7893 - val_loss: 0.4612 - val_accuracy: 0.7877
Epoch 7/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4584 - accuracy: 0.7963 - val_loss: 0.4427 - val_accuracy: 0.7877
Epoch 8/230
8/8 [========

8/8 [==============================] - 0s 2ms/step - loss: 0.3251 - accuracy: 0.8750 - val_loss: 0.4961 - val_accuracy: 0.7989
Epoch 60/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3241 - accuracy: 0.8792 - val_loss: 0.5028 - val_accuracy: 0.7989
Epoch 61/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3237 - accuracy: 0.8792 - val_loss: 0.5064 - val_accuracy: 0.8045
Epoch 62/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3221 - accuracy: 0.8792 - val_loss: 0.5122 - val_accuracy: 0.7989
Epoch 63/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3221 - accuracy: 0.8778 - val_loss: 0.5124 - val_accuracy: 0.7989
Epoch 64/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3229 - accuracy: 0.8778 - val_loss: 0.5129 - val_accuracy: 0.7989
Epoch 65/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3211 - accuracy: 0.8792 - val_loss: 0.5146 - val_accuracy: 0.7989
Epoch 66/230
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.2936 - accuracy: 0.8862 - val_loss: 0.6282 - val_accuracy: 0.7933
Epoch 118/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2924 - accuracy: 0.8890 - val_loss: 0.6316 - val_accuracy: 0.7933
Epoch 119/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2924 - accuracy: 0.8876 - val_loss: 0.6295 - val_accuracy: 0.7821
Epoch 120/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2932 - accuracy: 0.8890 - val_loss: 0.6366 - val_accuracy: 0.7933
Epoch 121/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2928 - accuracy: 0.8876 - val_loss: 0.6309 - val_accuracy: 0.7877
Epoch 122/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2936 - accuracy: 0.8820 - val_loss: 0.6456 - val_accuracy: 0.7933
Epoch 123/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2938 - accuracy: 0.8904 - val_loss: 0.6354 - val_accuracy: 0.7821
Epoch 124/230
8/8 [========

Epoch 175/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2809 - accuracy: 0.8919 - val_loss: 0.7468 - val_accuracy: 0.7877
Epoch 176/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2787 - accuracy: 0.8919 - val_loss: 0.7509 - val_accuracy: 0.7877
Epoch 177/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2822 - accuracy: 0.8919 - val_loss: 0.7486 - val_accuracy: 0.7821
Epoch 178/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2801 - accuracy: 0.8862 - val_loss: 0.7535 - val_accuracy: 0.7821
Epoch 179/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2838 - accuracy: 0.8890 - val_loss: 0.7600 - val_accuracy: 0.7877
Epoch 180/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2808 - accuracy: 0.8890 - val_loss: 0.7543 - val_accuracy: 0.7765
Epoch 181/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2830 - accuracy: 0.8919 - val_loss: 0.7633 - val_accuracy: 0.7933
Epoch 182/230

In [62]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,
                          input_dim=44,
                          kernel_initializer='normal',
                          activation='relu'),
    tf.keras.layers.Dense(88, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=230, batch_size=92, validation_data=(x_test, y_test), verbose=1)

Epoch 1/230
8/8 [==============================] - 0s 15ms/step - loss: 0.6874 - accuracy: 0.5702 - val_loss: 0.6708 - val_accuracy: 0.6983
Epoch 2/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6627 - accuracy: 0.7135 - val_loss: 0.6347 - val_accuracy: 0.7207
Epoch 3/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6254 - accuracy: 0.7247 - val_loss: 0.5829 - val_accuracy: 0.7542
Epoch 4/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5713 - accuracy: 0.7781 - val_loss: 0.5234 - val_accuracy: 0.7989
Epoch 5/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5154 - accuracy: 0.7992 - val_loss: 0.4695 - val_accuracy: 0.8101
Epoch 6/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4685 - accuracy: 0.8244 - val_loss: 0.4353 - val_accuracy: 0.8212
Epoch 7/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4420 - accuracy: 0.8216 - val_loss: 0.4199 - val_accuracy: 0.8268
Epoch 8/230
8/8 [=========

8/8 [==============================] - 0s 2ms/step - loss: 0.3409 - accuracy: 0.8638 - val_loss: 0.4729 - val_accuracy: 0.7877
Epoch 60/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3402 - accuracy: 0.8638 - val_loss: 0.4748 - val_accuracy: 0.7989
Epoch 61/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3378 - accuracy: 0.8652 - val_loss: 0.4785 - val_accuracy: 0.7989
Epoch 62/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3379 - accuracy: 0.8638 - val_loss: 0.4805 - val_accuracy: 0.7989
Epoch 63/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3393 - accuracy: 0.8638 - val_loss: 0.4826 - val_accuracy: 0.7989
Epoch 64/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3385 - accuracy: 0.8638 - val_loss: 0.4845 - val_accuracy: 0.7933
Epoch 65/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3361 - accuracy: 0.8652 - val_loss: 0.4835 - val_accuracy: 0.7821
Epoch 66/230
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.3106 - accuracy: 0.8750 - val_loss: 0.5519 - val_accuracy: 0.7709
Epoch 118/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3107 - accuracy: 0.8764 - val_loss: 0.5622 - val_accuracy: 0.7821
Epoch 119/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3137 - accuracy: 0.8750 - val_loss: 0.5659 - val_accuracy: 0.7933
Epoch 120/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3134 - accuracy: 0.8778 - val_loss: 0.5629 - val_accuracy: 0.7933
Epoch 121/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3146 - accuracy: 0.8778 - val_loss: 0.5543 - val_accuracy: 0.7654
Epoch 122/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3155 - accuracy: 0.8778 - val_loss: 0.5577 - val_accuracy: 0.7709
Epoch 123/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3095 - accuracy: 0.8792 - val_loss: 0.5577 - val_accuracy: 0.7765
Epoch 124/230
8/8 [========

Epoch 175/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2988 - accuracy: 0.8820 - val_loss: 0.5999 - val_accuracy: 0.7598
Epoch 176/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2985 - accuracy: 0.8806 - val_loss: 0.6006 - val_accuracy: 0.7654
Epoch 177/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2974 - accuracy: 0.8848 - val_loss: 0.6011 - val_accuracy: 0.7598
Epoch 178/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2990 - accuracy: 0.8806 - val_loss: 0.6044 - val_accuracy: 0.7709
Epoch 179/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2990 - accuracy: 0.8806 - val_loss: 0.6018 - val_accuracy: 0.7598
Epoch 180/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2971 - accuracy: 0.8806 - val_loss: 0.6038 - val_accuracy: 0.7598
Epoch 181/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2966 - accuracy: 0.8792 - val_loss: 0.6052 - val_accuracy: 0.7598
Epoch 182/230

In [63]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,
                          input_dim=44,
                          kernel_initializer='normal',
                          activation='relu'),
    tf.keras.layers.Dense(88, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=230, batch_size=92, validation_data=(x_test, y_test), verbose=1)

Epoch 1/230
8/8 [==============================] - 0s 14ms/step - loss: 0.6922 - accuracy: 0.5618 - val_loss: 0.6887 - val_accuracy: 0.6145
Epoch 2/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6854 - accuracy: 0.6194 - val_loss: 0.6822 - val_accuracy: 0.6257
Epoch 3/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6798 - accuracy: 0.6138 - val_loss: 0.6765 - val_accuracy: 0.6257
Epoch 4/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6750 - accuracy: 0.6138 - val_loss: 0.6713 - val_accuracy: 0.6257
Epoch 5/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6704 - accuracy: 0.6138 - val_loss: 0.6668 - val_accuracy: 0.6257
Epoch 6/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6664 - accuracy: 0.6138 - val_loss: 0.6629 - val_accuracy: 0.6257
Epoch 7/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6630 - accuracy: 0.6138 - val_loss: 0.6594 - val_accuracy: 0.6257
Epoch 8/230
8/8 [=========

8/8 [==============================] - 0s 2ms/step - loss: 0.4883 - accuracy: 0.8006 - val_loss: 0.4799 - val_accuracy: 0.7989
Epoch 60/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4863 - accuracy: 0.7963 - val_loss: 0.4777 - val_accuracy: 0.8045
Epoch 61/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4844 - accuracy: 0.7963 - val_loss: 0.4755 - val_accuracy: 0.8156
Epoch 62/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4821 - accuracy: 0.7978 - val_loss: 0.4734 - val_accuracy: 0.8212
Epoch 63/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4801 - accuracy: 0.8006 - val_loss: 0.4714 - val_accuracy: 0.8156
Epoch 64/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4783 - accuracy: 0.7963 - val_loss: 0.4694 - val_accuracy: 0.8156
Epoch 65/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4767 - accuracy: 0.7949 - val_loss: 0.4675 - val_accuracy: 0.8156
Epoch 66/230
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.4305 - accuracy: 0.8160 - val_loss: 0.4144 - val_accuracy: 0.8324
Epoch 118/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4298 - accuracy: 0.8146 - val_loss: 0.4139 - val_accuracy: 0.8324
Epoch 119/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4291 - accuracy: 0.8174 - val_loss: 0.4132 - val_accuracy: 0.8324
Epoch 120/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4292 - accuracy: 0.8174 - val_loss: 0.4128 - val_accuracy: 0.8324
Epoch 121/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4286 - accuracy: 0.8174 - val_loss: 0.4125 - val_accuracy: 0.8324
Epoch 122/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4280 - accuracy: 0.8174 - val_loss: 0.4120 - val_accuracy: 0.8324
Epoch 123/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4276 - accuracy: 0.8174 - val_loss: 0.4115 - val_accuracy: 0.8324
Epoch 124/230
8/8 [========

8/8 [==============================] - 0s 2ms/step - loss: 0.4129 - accuracy: 0.8329 - val_loss: 0.3969 - val_accuracy: 0.8324
Epoch 176/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4126 - accuracy: 0.8343 - val_loss: 0.3966 - val_accuracy: 0.8324
Epoch 177/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4126 - accuracy: 0.8329 - val_loss: 0.3965 - val_accuracy: 0.8324
Epoch 178/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4124 - accuracy: 0.8343 - val_loss: 0.3963 - val_accuracy: 0.8324
Epoch 179/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4121 - accuracy: 0.8343 - val_loss: 0.3962 - val_accuracy: 0.8324
Epoch 180/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4120 - accuracy: 0.8343 - val_loss: 0.3961 - val_accuracy: 0.8324
Epoch 181/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4119 - accuracy: 0.8315 - val_loss: 0.3958 - val_accuracy: 0.8324
Epoch 182/230
8/8 [========

In [64]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,
                          input_dim=44,
                          kernel_initializer='normal',
                          activation='relu'),
    tf.keras.layers.Dense(88, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='RMSprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=230, batch_size=92, validation_data=(x_test, y_test), verbose=1)

Epoch 1/230
8/8 [==============================] - 0s 14ms/step - loss: 0.6758 - accuracy: 0.6081 - val_loss: 0.6487 - val_accuracy: 0.6592
Epoch 2/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6314 - accuracy: 0.6840 - val_loss: 0.6011 - val_accuracy: 0.7263
Epoch 3/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5815 - accuracy: 0.7416 - val_loss: 0.5516 - val_accuracy: 0.8045
Epoch 4/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5333 - accuracy: 0.8090 - val_loss: 0.5056 - val_accuracy: 0.7933
Epoch 5/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4945 - accuracy: 0.8104 - val_loss: 0.4699 - val_accuracy: 0.8101
Epoch 6/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4635 - accuracy: 0.8160 - val_loss: 0.4474 - val_accuracy: 0.8268
Epoch 7/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4432 - accuracy: 0.8174 - val_loss: 0.4287 - val_accuracy: 0.8212
Epoch 8/230
8/8 [=========

8/8 [==============================] - 0s 2ms/step - loss: 0.3480 - accuracy: 0.8680 - val_loss: 0.4511 - val_accuracy: 0.8045
Epoch 60/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3444 - accuracy: 0.8652 - val_loss: 0.4629 - val_accuracy: 0.7877
Epoch 61/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3469 - accuracy: 0.8610 - val_loss: 0.4621 - val_accuracy: 0.7821
Epoch 62/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3440 - accuracy: 0.8694 - val_loss: 0.4570 - val_accuracy: 0.7933
Epoch 63/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3473 - accuracy: 0.8694 - val_loss: 0.4592 - val_accuracy: 0.7933
Epoch 64/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3427 - accuracy: 0.8666 - val_loss: 0.4594 - val_accuracy: 0.7933
Epoch 65/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3450 - accuracy: 0.8694 - val_loss: 0.4716 - val_accuracy: 0.7877
Epoch 66/230
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.3144 - accuracy: 0.8792 - val_loss: 0.5359 - val_accuracy: 0.7821
Epoch 118/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3151 - accuracy: 0.8792 - val_loss: 0.5390 - val_accuracy: 0.7709
Epoch 119/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3184 - accuracy: 0.8722 - val_loss: 0.5390 - val_accuracy: 0.7821
Epoch 120/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3159 - accuracy: 0.8764 - val_loss: 0.5426 - val_accuracy: 0.7821
Epoch 121/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3157 - accuracy: 0.8750 - val_loss: 0.5560 - val_accuracy: 0.7933
Epoch 122/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3128 - accuracy: 0.8792 - val_loss: 0.5519 - val_accuracy: 0.7877
Epoch 123/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3123 - accuracy: 0.8806 - val_loss: 0.5528 - val_accuracy: 0.7877
Epoch 124/230
8/8 [========

8/8 [==============================] - 0s 2ms/step - loss: 0.2997 - accuracy: 0.8820 - val_loss: 0.6417 - val_accuracy: 0.7933
Epoch 176/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2987 - accuracy: 0.8890 - val_loss: 0.6252 - val_accuracy: 0.7654
Epoch 177/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2977 - accuracy: 0.8806 - val_loss: 0.6325 - val_accuracy: 0.7877
Epoch 178/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2950 - accuracy: 0.8862 - val_loss: 0.6287 - val_accuracy: 0.7765
Epoch 179/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3033 - accuracy: 0.8820 - val_loss: 0.6271 - val_accuracy: 0.7709
Epoch 180/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3059 - accuracy: 0.8820 - val_loss: 0.6288 - val_accuracy: 0.7765
Epoch 181/230
8/8 [==============================] - 0s 2ms/step - loss: 0.2976 - accuracy: 0.8806 - val_loss: 0.6292 - val_accuracy: 0.7654
Epoch 182/230
8/8 [========

In [65]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,
                          input_dim=44,
                          kernel_initializer='normal',
                          activation='relu'),
    tf.keras.layers.Dense(88, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='Adadelta', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=230, batch_size=92, validation_data=(x_test, y_test), verbose=1)

Epoch 1/230
8/8 [==============================] - 0s 15ms/step - loss: 0.6918 - accuracy: 0.5463 - val_loss: 0.6917 - val_accuracy: 0.5307
Epoch 2/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6918 - accuracy: 0.5660 - val_loss: 0.6916 - val_accuracy: 0.5642
Epoch 3/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6917 - accuracy: 0.5899 - val_loss: 0.6915 - val_accuracy: 0.5642
Epoch 4/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.5913 - val_loss: 0.6915 - val_accuracy: 0.5642
Epoch 5/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.5913 - val_loss: 0.6914 - val_accuracy: 0.5642
Epoch 6/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6915 - accuracy: 0.5913 - val_loss: 0.6914 - val_accuracy: 0.5642
Epoch 7/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6914 - accuracy: 0.5913 - val_loss: 0.6913 - val_accuracy: 0.5698
Epoch 8/230
8/8 [=========

8/8 [==============================] - 0s 2ms/step - loss: 0.6879 - accuracy: 0.6278 - val_loss: 0.6876 - val_accuracy: 0.5978
Epoch 60/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6878 - accuracy: 0.6278 - val_loss: 0.6876 - val_accuracy: 0.5978
Epoch 61/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6877 - accuracy: 0.6278 - val_loss: 0.6875 - val_accuracy: 0.5922
Epoch 62/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6876 - accuracy: 0.6348 - val_loss: 0.6874 - val_accuracy: 0.5922
Epoch 63/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6876 - accuracy: 0.6362 - val_loss: 0.6873 - val_accuracy: 0.5978
Epoch 64/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6875 - accuracy: 0.6362 - val_loss: 0.6872 - val_accuracy: 0.5978
Epoch 65/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6874 - accuracy: 0.6376 - val_loss: 0.6872 - val_accuracy: 0.5978
Epoch 66/230
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.6834 - accuracy: 0.6376 - val_loss: 0.6829 - val_accuracy: 0.6201
Epoch 118/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6833 - accuracy: 0.6376 - val_loss: 0.6828 - val_accuracy: 0.6201
Epoch 119/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6832 - accuracy: 0.6376 - val_loss: 0.6827 - val_accuracy: 0.6201
Epoch 120/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6831 - accuracy: 0.6376 - val_loss: 0.6827 - val_accuracy: 0.6201
Epoch 121/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6831 - accuracy: 0.6376 - val_loss: 0.6826 - val_accuracy: 0.6201
Epoch 122/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6830 - accuracy: 0.6376 - val_loss: 0.6825 - val_accuracy: 0.6201
Epoch 123/230
8/8 [==============================] - 0s 3ms/step - loss: 0.6829 - accuracy: 0.6376 - val_loss: 0.6824 - val_accuracy: 0.6201
Epoch 124/230
8/8 [========

Epoch 175/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6785 - accuracy: 0.6236 - val_loss: 0.6777 - val_accuracy: 0.6201
Epoch 176/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6784 - accuracy: 0.6236 - val_loss: 0.6776 - val_accuracy: 0.6201
Epoch 177/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6783 - accuracy: 0.6250 - val_loss: 0.6775 - val_accuracy: 0.6201
Epoch 178/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6782 - accuracy: 0.6236 - val_loss: 0.6774 - val_accuracy: 0.6201
Epoch 179/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6781 - accuracy: 0.6222 - val_loss: 0.6773 - val_accuracy: 0.6201
Epoch 180/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6780 - accuracy: 0.6222 - val_loss: 0.6772 - val_accuracy: 0.6201
Epoch 181/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6779 - accuracy: 0.6222 - val_loss: 0.6771 - val_accuracy: 0.6145
Epoch 182/230

In [66]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,
                          input_dim=44,
                          kernel_initializer='normal',
                          activation='relu'),
    tf.keras.layers.Dense(88, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='Adagrad', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=230, batch_size=92, validation_data=(x_test, y_test), verbose=1)

Epoch 1/230
8/8 [==============================] - 0s 37ms/step - loss: 0.6973 - accuracy: 0.3947 - val_loss: 0.6934 - val_accuracy: 0.3743
Epoch 2/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6932 - accuracy: 0.3989 - val_loss: 0.6894 - val_accuracy: 0.3911
Epoch 3/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6895 - accuracy: 0.4452 - val_loss: 0.6858 - val_accuracy: 0.5251
Epoch 4/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6860 - accuracy: 0.6152 - val_loss: 0.6820 - val_accuracy: 0.7374
Epoch 5/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6825 - accuracy: 0.7303 - val_loss: 0.6784 - val_accuracy: 0.7654
Epoch 6/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6792 - accuracy: 0.7317 - val_loss: 0.6749 - val_accuracy: 0.7821
Epoch 7/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6761 - accuracy: 0.7303 - val_loss: 0.6718 - val_accuracy: 0.7821
Epoch 8/230
8/8 [=========

8/8 [==============================] - 0s 2ms/step - loss: 0.6014 - accuracy: 0.7233 - val_loss: 0.5919 - val_accuracy: 0.7486
Epoch 58/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6002 - accuracy: 0.7233 - val_loss: 0.5908 - val_accuracy: 0.7486
Epoch 59/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5991 - accuracy: 0.7233 - val_loss: 0.5896 - val_accuracy: 0.7486
Epoch 60/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5980 - accuracy: 0.7261 - val_loss: 0.5885 - val_accuracy: 0.7486
Epoch 61/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5969 - accuracy: 0.7261 - val_loss: 0.5873 - val_accuracy: 0.7486
Epoch 62/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5958 - accuracy: 0.7261 - val_loss: 0.5862 - val_accuracy: 0.7486
Epoch 63/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5947 - accuracy: 0.7289 - val_loss: 0.5850 - val_accuracy: 0.7486
Epoch 64/230
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.5438 - accuracy: 0.7851 - val_loss: 0.5325 - val_accuracy: 0.7877
Epoch 116/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5430 - accuracy: 0.7851 - val_loss: 0.5316 - val_accuracy: 0.7877
Epoch 117/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5421 - accuracy: 0.7851 - val_loss: 0.5307 - val_accuracy: 0.7821
Epoch 118/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5413 - accuracy: 0.7893 - val_loss: 0.5298 - val_accuracy: 0.7821
Epoch 119/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5404 - accuracy: 0.7893 - val_loss: 0.5289 - val_accuracy: 0.7821
Epoch 120/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5395 - accuracy: 0.7893 - val_loss: 0.5280 - val_accuracy: 0.7821
Epoch 121/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5387 - accuracy: 0.7893 - val_loss: 0.5271 - val_accuracy: 0.7821
Epoch 122/230
8/8 [========

8/8 [==============================] - 0s 2ms/step - loss: 0.4996 - accuracy: 0.8062 - val_loss: 0.4874 - val_accuracy: 0.8268
Epoch 174/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4990 - accuracy: 0.8090 - val_loss: 0.4867 - val_accuracy: 0.8268
Epoch 175/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4983 - accuracy: 0.8062 - val_loss: 0.4861 - val_accuracy: 0.8268
Epoch 176/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4977 - accuracy: 0.8062 - val_loss: 0.4855 - val_accuracy: 0.8268
Epoch 177/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4970 - accuracy: 0.8062 - val_loss: 0.4848 - val_accuracy: 0.8268
Epoch 178/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4964 - accuracy: 0.8062 - val_loss: 0.4842 - val_accuracy: 0.8268
Epoch 179/230
8/8 [==============================] - 0s 2ms/step - loss: 0.4958 - accuracy: 0.8076 - val_loss: 0.4836 - val_accuracy: 0.8268
Epoch 180/230
8/8 [========

In [67]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,
                          input_dim=44,
                          kernel_initializer='normal',
                          activation='relu'),
    tf.keras.layers.Dense(88, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='Adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=230, batch_size=92, validation_data=(x_test, y_test), verbose=1)

Epoch 1/230
8/8 [==============================] - 0s 14ms/step - loss: 0.6912 - accuracy: 0.5449 - val_loss: 0.6756 - val_accuracy: 0.6927
Epoch 2/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6661 - accuracy: 0.6334 - val_loss: 0.6520 - val_accuracy: 0.6313
Epoch 3/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6445 - accuracy: 0.6166 - val_loss: 0.6308 - val_accuracy: 0.6313
Epoch 4/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6259 - accuracy: 0.6180 - val_loss: 0.6089 - val_accuracy: 0.6536
Epoch 5/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6045 - accuracy: 0.6278 - val_loss: 0.5880 - val_accuracy: 0.6816
Epoch 6/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5836 - accuracy: 0.6756 - val_loss: 0.5656 - val_accuracy: 0.7207
Epoch 7/230
8/8 [==============================] - 0s 2ms/step - loss: 0.5610 - accuracy: 0.7360 - val_loss: 0.5431 - val_accuracy: 0.7263
Epoch 8/230
8/8 [=========

8/8 [==============================] - 0s 2ms/step - loss: 0.3677 - accuracy: 0.8567 - val_loss: 0.4073 - val_accuracy: 0.8045
Epoch 60/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3669 - accuracy: 0.8511 - val_loss: 0.4076 - val_accuracy: 0.8101
Epoch 61/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3668 - accuracy: 0.8525 - val_loss: 0.4076 - val_accuracy: 0.8101
Epoch 62/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3657 - accuracy: 0.8539 - val_loss: 0.4106 - val_accuracy: 0.8101
Epoch 63/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3651 - accuracy: 0.8581 - val_loss: 0.4110 - val_accuracy: 0.8101
Epoch 64/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3649 - accuracy: 0.8567 - val_loss: 0.4119 - val_accuracy: 0.8045
Epoch 65/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3640 - accuracy: 0.8553 - val_loss: 0.4122 - val_accuracy: 0.8045
Epoch 66/230
8/8 [===============

8/8 [==============================] - 0s 2ms/step - loss: 0.3399 - accuracy: 0.8652 - val_loss: 0.4514 - val_accuracy: 0.8045
Epoch 118/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3394 - accuracy: 0.8638 - val_loss: 0.4523 - val_accuracy: 0.8045
Epoch 119/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3390 - accuracy: 0.8652 - val_loss: 0.4540 - val_accuracy: 0.8045
Epoch 120/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3389 - accuracy: 0.8638 - val_loss: 0.4538 - val_accuracy: 0.8045
Epoch 121/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3382 - accuracy: 0.8652 - val_loss: 0.4544 - val_accuracy: 0.8045
Epoch 122/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3381 - accuracy: 0.8652 - val_loss: 0.4567 - val_accuracy: 0.7989
Epoch 123/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3376 - accuracy: 0.8666 - val_loss: 0.4561 - val_accuracy: 0.8045
Epoch 124/230
8/8 [========

8/8 [==============================] - 0s 2ms/step - loss: 0.3209 - accuracy: 0.8778 - val_loss: 0.4875 - val_accuracy: 0.7877
Epoch 176/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3208 - accuracy: 0.8778 - val_loss: 0.4891 - val_accuracy: 0.7933
Epoch 177/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3211 - accuracy: 0.8778 - val_loss: 0.4886 - val_accuracy: 0.7989
Epoch 178/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3196 - accuracy: 0.8764 - val_loss: 0.4919 - val_accuracy: 0.7933
Epoch 179/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3204 - accuracy: 0.8764 - val_loss: 0.4913 - val_accuracy: 0.7933
Epoch 180/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3193 - accuracy: 0.8764 - val_loss: 0.4911 - val_accuracy: 0.7877
Epoch 181/230
8/8 [==============================] - 0s 2ms/step - loss: 0.3191 - accuracy: 0.8778 - val_loss: 0.4922 - val_accuracy: 0.7933
Epoch 182/230
8/8 [========

In [68]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(20,
                          input_dim=44,
                          kernel_initializer='normal',
                          activation='relu'),
    tf.keras.layers.Dense(88, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])
model.compile(optimizer='Ftrl', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=230, batch_size=92, validation_data=(x_test, y_test), verbose=1)

Epoch 1/230
8/8 [==============================] - 0s 15ms/step - loss: 0.6925 - accuracy: 0.5941 - val_loss: 0.6926 - val_accuracy: 0.6257
Epoch 2/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6925 - accuracy: 0.6138 - val_loss: 0.6922 - val_accuracy: 0.6257
Epoch 3/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6921 - accuracy: 0.6138 - val_loss: 0.6918 - val_accuracy: 0.6257
Epoch 4/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6918 - accuracy: 0.6138 - val_loss: 0.6915 - val_accuracy: 0.6257
Epoch 5/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6916 - accuracy: 0.6138 - val_loss: 0.6912 - val_accuracy: 0.6257
Epoch 6/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6913 - accuracy: 0.6138 - val_loss: 0.6910 - val_accuracy: 0.6257
Epoch 7/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6911 - accuracy: 0.6138 - val_loss: 0.6908 - val_accuracy: 0.6257
Epoch 8/230
8/8 [=========

Epoch 58/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6861 - accuracy: 0.6138 - val_loss: 0.6853 - val_accuracy: 0.6257
Epoch 59/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6860 - accuracy: 0.6138 - val_loss: 0.6852 - val_accuracy: 0.6257
Epoch 60/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6860 - accuracy: 0.6138 - val_loss: 0.6851 - val_accuracy: 0.6257
Epoch 61/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6859 - accuracy: 0.6138 - val_loss: 0.6851 - val_accuracy: 0.6257
Epoch 62/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6859 - accuracy: 0.6138 - val_loss: 0.6850 - val_accuracy: 0.6257
Epoch 63/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6858 - accuracy: 0.6138 - val_loss: 0.6849 - val_accuracy: 0.6257
Epoch 64/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6857 - accuracy: 0.6138 - val_loss: 0.6849 - val_accuracy: 0.6257
Epoch 65/230
8/8 [==

8/8 [==============================] - 0s 2ms/step - loss: 0.6833 - accuracy: 0.6138 - val_loss: 0.6821 - val_accuracy: 0.6257
Epoch 117/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6833 - accuracy: 0.6138 - val_loss: 0.6821 - val_accuracy: 0.6257
Epoch 118/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6832 - accuracy: 0.6138 - val_loss: 0.6821 - val_accuracy: 0.6257
Epoch 119/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6832 - accuracy: 0.6138 - val_loss: 0.6820 - val_accuracy: 0.6257
Epoch 120/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6832 - accuracy: 0.6138 - val_loss: 0.6820 - val_accuracy: 0.6257
Epoch 121/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6831 - accuracy: 0.6138 - val_loss: 0.6819 - val_accuracy: 0.6257
Epoch 122/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6831 - accuracy: 0.6138 - val_loss: 0.6819 - val_accuracy: 0.6257
Epoch 123/230
8/8 [========

8/8 [==============================] - 0s 2ms/step - loss: 0.6814 - accuracy: 0.6138 - val_loss: 0.6800 - val_accuracy: 0.6257
Epoch 175/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6814 - accuracy: 0.6138 - val_loss: 0.6799 - val_accuracy: 0.6257
Epoch 176/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.6138 - val_loss: 0.6799 - val_accuracy: 0.6257
Epoch 177/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.6138 - val_loss: 0.6799 - val_accuracy: 0.6257
Epoch 178/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.6138 - val_loss: 0.6798 - val_accuracy: 0.6257
Epoch 179/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6813 - accuracy: 0.6138 - val_loss: 0.6798 - val_accuracy: 0.6257
Epoch 180/230
8/8 [==============================] - 0s 2ms/step - loss: 0.6812 - accuracy: 0.6138 - val_loss: 0.6798 - val_accuracy: 0.6257
Epoch 181/230
8/8 [========